# Session 3: Signal analysis

In [193]:
import numpy as np
from numpy import histogram
import soundfile as sf
import sounddevice as sd
from scipy.signal import correlate, correlation_lags
import matplotlib.pyplot as plt
import numpy.fft as npf
import PyQt5
%matplotlib qt


## Exercise 3.1

In [194]:
noise_length = 1000000
x1 = np.random.rand(noise_length)*4
x2 = np.random.randn(noise_length)*0.71+2
x3 = np.random.randn(noise_length)*1.23+2
print('x1 mean '+str(np.mean(x1))+' and var '+str(np.var(x1)))
print('x2 mean '+str(np.mean(x2))+' and var '+str(np.var(x2)))
print('x3 mean '+str(np.mean(x3))+' and var '+str(np.var(x3)))

x1 mean 2.00151554028431 and var 1.33196320809315
x2 mean 1.999651710065917 and var 0.5041343541932136
x3 mean 2.000434042722431 and var 1.5148468444173324


In [195]:
edges = np.arange(min(np.min(x3), np.min(x2), np.min(x1)), max(np.max(x1), np.max(x2), np.max(x3)), 0.1)
hx1, _ = np.histogram(x1, edges, density=True)
hx2, _ = np.histogram(x2, edges, density=True)
hx3, _ = np.histogram(x3, edges, density=True)
centers = 0.5*(edges[1:] + edges[:-1])
plt.plot(centers, hx1, label='x1')
plt.plot(centers, hx2, label='x2')
plt.plot(centers, hx3, label='x3')
plt.legend()
plt.xlabel('value')
plt.ylabel('density')

Text(0, 0.5, 'density')

In [209]:
c1 = np.cumsum(hx1)/sum(hx1)
c2 = np.cumsum(hx2)/sum(hx2)
c3 = np.cumsum(hx3)/sum(hx3)
plt.plot(centers, c1, label='c1')
plt.plot(centers, c2, label='c2')
plt.plot(centers, c3, label='c3')
plt.legend()
plt.xlabel('value')
plt.ylabel('cumulated density')
print(1-c1[abs(centers-1)<0.05])
print(1-c2[abs(centers-1)<0.05])
print(1-c3[abs(centers-1)<0.05])

[0.727841]
[0.899389]
[0.76968577]


## Exercise 3.2

In [173]:
data, sr = sf.read('voice1.wav')
data_length = len(data)
length_ms = data_length/sr*100
print('the file has a length of %3.2f ms with a sr of %i' % (length_ms, sr))
# sd.play(data, sr)
# status = sd.wait()
phi_xx = correlate(data, data)
lags = correlation_lags(data_length, data_length)
plt.plot(lags, phi_xx)
print('f is %4.2f'%(sr/50))

the file has a length of 10.00 ms with a sr of 8000
f is 160.00


In [174]:
f = np.load('sequences.npz')
x = f['x']
y1 = f['y1']
y2 = f['y2']
y3 = f['y3']
phi_xy1 = correlate(x, y1)
phi_xy2 = correlate(x, y2)
phi_xy3 = correlate(x, y3)
lags1 = correlation_lags(len(x), len(y1))
plt.plot(phi_xy1)
plt.show()
lags2 = correlation_lags(len(x), len(y2))
plt.plot(lags2, phi_xy2)
plt.show()
lags3 = correlation_lags(len(x), len(y3))
plt.plot(lags3, phi_xy3)
plt.show()
print('the scaling factor of y2 is %3.2f'%(max(y2)/max(x)))
print('the scaling factor of y3 is %3.2f'%(max(y3)/max(x)))

indice_maximum2=np.argmax(phi_xy2)
indice_maximum3=np.argmax(phi_xy3)

lags_shift_2=lags2[indice_maximum2]
lags_shift_3=lags3[indice_maximum3]
print("y2 lags shift",lags_shift_2," y3 lags shift",lags_shift_3)

the scaling factor of y2 is 1.00
the scaling factor of y3 is 2.00
y2 lags shift 14  y3 lags shift 0


## Exercise 3.3

In [177]:
N = 64

x0 = np.zeros((N))
x0[0] = 1
x1 = np.zeros((N))
x1[1] = 1
x2 = np.zeros((N))
x2[2] = 1

# freq=np.fft.fftfreq(N)

def show_fft(signal):


    fft_signal = np.fft.fft(signal)

    fft_real = fft_signal.real
    fft_imag = fft_signal.imag
    indice_maximum = np.argmax(fft_signal)
    magnitude = np.abs(fft_signal)
    phase = np.angle(fft_signal)
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    ax1.plot(fft_imag,label="Imaginary part ")
    ax2.plot(fft_real,label="Real part")
    ax3.plot(magnitude, label="magnitude ")
    ax4.plot(phase, label="phase")
    ax1.set_title("Imaginary part")
    ax2.set_title("Real part")
    ax3.set_title("magnitude")
    ax4.set_title("phase")
    plt.show()


show_fft(x0)
# print("magnitude1 is:", magnitude1," phase1 is:", phase1)

show_fft(x1)
# print("magnitude2 is:", magnitude2," phase2 is:", phase2)

show_fft(x2)
# print("magnitude3 is:", magnitude3," phase3 is:", phase3)

In [178]:
N = 42
n = 3
k = np.arange(0, N)
x_even = np.cos(k*2*np.pi*n/N)
show_fft(x_even)
n = 10.33
x_odd = np.cos(k*2*np.pi*n/N)
show_fft(x_odd)
k

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41])

## Exercise 3.4

In [182]:
data, sr = sf.read('distorted.wav')
fft_of_data = npf.rfft(data)
n = data.size
freq = npf.rfftfreq(n)*sr
plt.plot(freq, fft_of_data)
plt.legend()
plt.xlabel('frequency')
plt.ylabel('rfft')
plt.show()

d:\programme\python\lib\site-packages\numpy\core\_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
No handles with labels found to put in legend.


## Exercise 3.5

In [184]:
from scipy.signal import spectrogram

data, sr = sf.read('voice2.wav')
data_length = len(data)
length_ms = data_length/sr*100
print('the file has a length of %3.2f ms with a sr of %i' % (length_ms, sr))
sd.play(data, sr)
status = sd.wait()

def compare_specs(x, fs, params_default, **kwargs):
    fig, (ax1, ax2) = plt.subplots(2, 1)
    fbin, tframe, S = spectrogram(x, fs, **params_default, scaling='spectrum')
    ax1.pcolormesh(tframe, fbin, 20*np.log10(S), shading='auto')
#     fig.colorbar()
    ax1.set_xlabel('t [s]')
    ax1.set_ylabel('f [Hz]')
    fbin, tframe, S = spectrogram(x, fs, **{**params_default, **kwargs}, scaling='spectrum')
    ax2.pcolormesh(tframe, fbin, 20*np.log10(S), shading='auto')
    ax2.set_xlabel('t [s]')
    ax2.set_ylabel('f [Hz]')

the file has a length of 1080.00 ms with a sr of 16000


In [190]:
params_default=dict(nperseg=512, noverlap=0, nfft=None, window='rect')
new_params=dict(nperseg=2000, noverlap=0, nfft=None, window='blackman')

In [191]:
compare_specs(data, sr, params_default, **new_params)

<ipython-input-184-98cff355ec8c>:13: RuntimeWarning: divide by zero encountered in log10
  ax1.pcolormesh(tframe, fbin, 20*np.log10(S), shading='auto')
